In [6]:
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import pymc
from pymc import utils
from pymc.database import base

import glob

#plotting functions
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

variables = ['massbal', 'precfactor', 'tempchange', 'ddfsnow']  
vn_title_dict = {'massbal':'Mass Balance',                                                                      
                 'precfactor':'Precipitation Factor',                                                              
                 'tempchange':'Temperature Bias',                                                               
                 'ddfsnow':'DDF Snow'}
vn_label_dict = {'massbal':'Mass balance\n[mwea]',                                                                      
                 'precfactor':'Precipitation factor\n[-]',                                                              
                 'tempchange':'Temperature bias\n[degC]',                                                               
                 'ddfsnow':'DDFsnow\n[mwe $degC^{-1} d^{-1}$]'}
metric_title_dict = {'Gelman-Rubin':'Gelman-Rubin Statistic',
                     'MC Error': 'Monte Carlo Error',
                     'Effective N': 'Effective Sample Size'}
metrics = ['Gelman-Rubin', 'MC Error', 'Effective N']

suffix = '_trunc'
mcmc_output_netcdf_fp = '../MCMC_data/netcdf' + suffix + '/'
mcmc_output_figures_fp = '../MCMC_data/figures' + suffix + '/'
mcmc_output_tables_fp = '../MCMC_data/tables/'
mcmc_output_csv_fp = '../MCMC_data/csv' + suffix + '/'
mcmc_output_hist_fp = '../MCMC_data/hist' + suffix + '/'

In [23]:
def assessment_vs_chain_length(iters, region='all', burn=0, mean=False):
    """
    Plot gelman-rubin statistic, effective_n (autocorrelation with lag
    100) and markov chain error plots.

    Takes the output from the Markov Chain model and plots the results
    for the mass balance, temperature change, precipitation factor,
    and degree day factor of snow.  Also, outputs the plots associated
    with the model.

    Parameters
    ----------
    iters : int
        Number of iterations associated with the Markov Chain
    burn : list of ints
        List of burn in values to plot for Gelman-Rubin stats

    Returns
    -------
    .png files
        saves figure showing how assessment values change with
        number of mcmc iterations
    """

    # hard code some variable names (dirty solution)
    variables = ['massbal', 'precfactor', 'tempchange', 'ddfsnow']
    vn_title_dict = {'massbal':'Mass Balance',
                     'precfactor':'Precipitation Factor',
                     'tempchange':'Temperature Bias',
                     'ddfsnow':'DDF Snow'}
    metric_title_dict = {'Gelman-Rubin':'Gelman-Rubin Statistic',
                         'MC Error': 'Monte Carlo Error',
                         'Effective N': 'Effective Sample Size'}
    metrics = ['Gelman-Rubin', 'MC Error', 'Effective N']
    vn_label_dict = {'massbal':'[mwea]',
                     'precfactor':'[-]',
                     'tempchange':'[degC]',
                     'ddfsnow':'[mwe $degC^{-1} d^{-1}$]'}


    if mean:
        # find mean values
        # find all netcdf files (representing glaciers)
        if region == 'all':
            regions = ['13', '14', '15']
            filelist = []
            for reg in regions:
                filelist.extend(glob.glob(mcmc_output_netcdf_fp + str(reg) + '*.nc'))
        else:
            filelist = glob.glob(mcmc_output_netcdf_fp + str(region) + '*.nc')

        mean_dict = {}

        for vn in variables:
            mean_dict[vn] = []

        for netcdf in filelist:

            print(netcdf)

            try:
                # open dataset
                ds = xr.open_dataset(netcdf)

                for vn in variables:
                    mean_dict[vn].append(abs(np.mean(ds['mp_value'].sel(chain=0, mp=vn).values)))

            except:
                print('Error: ', netcdf)
                pass

        # calculate total mean
        for vn in variables:
            mean_dict[vn] = np.mean(mean_dict[vn])



    # hard code font sizes
    ticks=10
    suptitle=14
    title=10
    titley = 1.05
    label=10
    plotline=2
    plotline2=1
    legend=10
    figsize=(6.5, 9)
    dpi=100
    hspace=0.4
    wspace=0.4
    sup_y = 0.97
    nrows=4
    ncols=3
    num_stds=1
    alpha = 0.7
    s_alpha = 0.5
    cutoffstyle = '--'
    cutoffcolor = 'black'
    cutoffwidth = 1.5
    num_stds = 2
    #colors = ['#614684', '#ddc73e', '#ff6135']
    colors = ['#387ea0', '#fcb200', '#d20048']
    xmin = 0.025
    xmax = 0.975

    # bins and ticks
    bdict = {}
    tdict = {}

    plt.figure(figsize=figsize, dpi=dpi)
    plt.subplots_adjust(wspace=wspace, hspace=hspace)

    m_len = len(metrics)
    v_len = len(variables)

    # create subplot for each variable
    for v_count, vn in enumerate(variables):

        df_dict = {}
        df_dict['Effective N'] = pd.read_csv(mcmc_output_csv_fp + vn + '_effective_n_df.csv').drop('Unnamed: 0', 1)
        df_dict['MC Error'] = pd.read_csv(mcmc_output_csv_fp + vn + '_mc_error_df.csv').drop('Unnamed: 0', 1)
        df_dict['normalized MC Error'] = pd.read_csv(mcmc_output_csv_fp +
                vn + '_normalized_mc_error_df.csv').drop('Unnamed: 0', 1)
        df_dict['Gelman-Rubin'] = pd.read_csv(mcmc_output_csv_fp + vn + '_gelman_rubin_df.csv').drop('Unnamed: 0', 1)

        #create subplot for each metric
        k = 0
        for m_count, metric in enumerate(metrics):

            if metric == 'MC Error' and (vn == 'precfactor' or vn == 'ddfsnow'):
                df = df_dict['normalized ' + metric]#.drop('Unnamed: 0', 1)
            else:
                df = df_dict[metric]#.drop('Unnamed: 0', 1)
            x = df.columns.values.astype(int)
            u_quantile = df.quantile(q=0.95).values
            l_quantile = df.quantile(q=0.05).values
            mean = df.mean().values
            median = df.median().values
            std = df.std().values

            # plot histogram
            ax = plt.subplot(nrows, ncols, m_len*v_count+m_count+1)

            ax.plot(x, mean, alpha=alpha, color=colors[k])

            #plot error region
            #ax.fill_between(x, mean + (num_stds * std), mean - (num_stds * std), alpha=s_alpha)
            ax.fill_between(x, u_quantile, l_quantile, alpha=s_alpha, color=colors[k])

            # niceties
            if v_count == 0:
                plt.title(metric_title_dict[metric], fontsize=title, y=titley)

            ylabel = ''
            # axis labels
            #if metric == 'MC Error' and (vn == 'tempchange' or vn == 'massbal'):
            #    ylabel = metric + ' ' + vn_label_dict[vn]
            #elif metric == 'MC Error':
            #    ylabel = metric + ' [%]'
            #else:
            #    ylabel = metric

            if m_count == 0:
                ax.set_ylabel(vn_title_dict[vn], fontsize=label, labelpad=10)
            #else:
                #ax.set_ylabel(ylabel, fontsize=label)

            if metric == 'Effective N':
                ax.set_ylim(bottom=0, top=1000)
                ax.axhline(y=100, xmin=xmin, xmax=xmax, color=cutoffcolor, linestyle=cutoffstyle, linewidth=cutoffwidth)
                #ticks = [item for item in ax.get_yticklabels()]
                #print(ticks)
            if metric == 'Gelman-Rubin':
                ax.set_ylim(bottom=1.0, top=1.12)
                ax.axhline(y=1.1, xmin=xmin, xmax=xmax, color=cutoffcolor, linestyle=cutoffstyle, linewidth=cutoffwidth)
            if metric == 'MC Error' and (vn == 'precfactor' or vn == 'ddfsnow'):
                ax.axhline(y=5, xmin=xmin, xmax=xmax, color=cutoffcolor, linestyle=cutoffstyle, linewidth=cutoffwidth)

            ax.set_xticks(np.arange(0, 31000, 10000))
            
            majorLocator = MultipleLocator(10000)
            minorLocator = MultipleLocator(2000)
            
            ax.xaxis.set_major_locator(majorLocator)
            ax.xaxis.set_minor_locator(minorLocator)
            
            def format_func(value, tick_number):
                # find number of multiples of pi/2
                N = int(value)
                return str(int(N/1000))


            ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            #ax.xaxis.set_major_formatter(majorFormatter)
            
            if v_count==3:
                ax.set_xlabel('Steps [$10^3$]')

            k += 1

    # Save figure
    plt.savefig(mcmc_output_figures_fp + 'assessment_vs_chain_length.png',
                bbox_inches='tight')


In [24]:
iterations = np.arange(1000, 31000, 3000)
iterations = np.append(iterations, 30000)
assessment_vs_chain_length(iters=iterations, region='all', burn=0)

In [70]:
region_no = 15.02793
csv_path = '../DEMs/Shean_2018_0806/hma_mb_20180803_1229.csv'
ds_all = pd.read_csv(csv_path)                                                                                    
ds_all[ds_all['RGIId'] == region_no]['mb_mwea'].values[0]

-0.088000000000000009

In [72]:
ds_all[ds_all['RGIId'] == region_no]['mb_mwea_sigma'].values[0]

0.27800000000000002

In [71]:
ds_all

RGIId            x            y     z_med     z_p16     z_p84  \
0      15.05152  -145758.234  -820691.042  7832.993  7700.992  8033.181   
1      15.02793   307937.214  -927265.323  7687.041  7536.363  7853.120   
2      15.02790   305153.261  -927260.154  7682.833  7509.912  7753.357   
3      15.05153  -146725.010  -821804.519  7679.389  7494.941  7912.212   
4      15.02827   308784.358  -927669.239  7668.919  7483.279  7881.574   
5      15.02828   310683.424  -929789.103  7547.470  7338.876  7662.285   
6      14.19942  -927804.005   -35168.378  7485.054  7405.273  7666.429   
7      14.23897 -1157128.322   106023.793  7411.674  7335.080  7497.418   
8      15.05141   -44297.109  -837269.979  7330.808  6972.165  7474.107   
9      14.06796  -752458.804    18037.549  7274.240  7118.639  7574.369   
10     14.19948  -929689.476   -36435.036  7250.578  6934.409  7541.479   
11     15.02842   305050.087  -937226.623  7230.928  7100.846  7268.708   
12     15.04148   -36426.360  -850984.776  7224.337  7062.962  7441.059   
13     14.23883 -1157892.344   106248.274  7180.162  7063.572  7300.190   
14     15.02847   312443.725  -918591.884  7177.543  7102.099  7232.221   
15     15.02826   310040.438  -926972.189  7148.821  6783.335  7490.224   
16     15.02699   305802.163  -909353.700  7133.658  6970.463  7268.929   
17     15.02792   307694.727  -926705.650  7129.487  7032.873  7244.476   
18     15.02909   307345.232  -909316.358  7122.483  6998.120  7224.217   
19     14.23698 -1129188.378   135015.806  7110.820  6953.950  7183.859   
20     15.06976  -509335.956  -558023.651  7110.075  6803.101  7356.369   
21     14.06815  -744025.959     5066.961  7109.524  6775.615  7468.015   
22     14.23692 -1129006.228   134182.451  7108.889  7066.040  7210.404   
23     15.04811   -84269.296  -838775.661  7102.007  6792.720  7324.981   
24     14.23697 -1129963.640   134401.966  7082.274  6852.627  7151.609   
25     14.23882 -1159138.100   106859.311  7064.378  6873.943  7260.789   
26     14.23837 -1159424.000   107750.886  7011.867  6849.064  7197.453   
27     14.23691 -1128552.592   134372.990  7005.095  6907.219  7069.966   
28     15.07146  -508416.045  -558586.840  6989.457  6738.304  7124.327   
29     15.03475   161468.606  -890222.577  6985.648  6802.957  7186.916   
...         ...          ...          ...       ...       ...       ...   
70137  13.05605  -455886.959  1023141.193  3200.460  3054.447  3308.016   
70138  13.23117 -1313236.469   435043.874  3197.396  3152.757  3289.496   
70139  13.05604  -459994.934  1019968.396  3193.681  3115.878  3289.795   
70140  13.21993 -1175278.452   772132.374  3193.555  3123.839  3248.175   
70141  13.05588  -446374.070  1018300.346  3191.737  3120.466  3257.279   
70142  13.22028 -1162349.287   800838.425  3189.919  3118.266  3240.675   
70143  13.05755  -455424.568  1019044.217  3185.549  3068.919  3304.469   
70144  13.05608  -456986.898  1024844.422  3185.043  3103.816  3276.248   
70145  13.05603  -459322.244  1020129.466  3178.900  3123.430  3294.433   
70146  13.20532 -1201625.181   485967.746  3173.291  3083.452  3284.716   
70147  13.22139 -1190054.518   741579.173  3171.116  3103.941  3319.752   
70148  13.05601  -457779.254  1019727.808  3170.303  3113.698  3278.220   
70149  13.05614  -461113.707  1025047.418  3167.647  3096.618  3252.582   
70150  13.05600  -456986.036  1019208.374  3161.349  3085.049  3226.900   
70151  13.20372 -1201804.746   424219.963  3153.898  2999.412  3291.961   
70152  13.05598  -453461.420  1018372.868  3148.662  3089.130  3237.109   
70153  13.18298 -1198331.462   426709.109  3145.206  3046.854  3215.621   
70154  13.05602  -458559.798  1020016.813  3143.891  3077.519  3249.190   
70155  13.05606  -455817.243  1024118.571  3132.532  3078.987  3225.032   
70156  13.05611  -455980.263  1025687.227  3126.777  3041.484  3226.633   
70157  13.22035 -1173017.157   764569.206  3119.156  3075.628  3172.710   
70158  13.22808 -1243653.517   4682